# Feature Extraction

In [ ]:
!git clone -b Stable https://github.com/cvai-roig-lab/Net2Brain.git

In [ ]:
!pip install Net2Brain/.

__Net2Brain__ allows you to use one of over 600 Deep Neural Networks (DNNs) for your experiments comparing human brain activity with the activations of artificial neural networks. The DNNs in __Net2Brain__ are obtained from what we call different _netsets_, which are libraries that provide different pretrained models. 

__Net2Brain__ provides access to the following _netsets_:
- [Standard torchvision](https://pytorch.org/vision/stable/models.html) (`standard`).
This netset is a collection of the torchvision models including models for image classification, pixelwise semantic segmentation, object detection, instance segmentation, person keypoint detection, video classification, and optical flow.
- [Timm](https://github.com/rwightman/pytorch-image-models#models) (`timm`). 
A deep-learning library created by Ross Wightman that contains a collection of state-of-the-art computer vision models.
- [PyTorch Hub](https://pytorch.org/docs/stable/hub.html) (`pytorch`). 
These models are accessible through the torch.hub API and are trained for different visual tasks. They are not included in the torchvision module.
- [Unet](https://pytorch.org/hub/mateuszbuda_brain-segmentation-pytorch_unet/) (`unet`). 
Unet also is available through the torch.hub.API and is trained for abnormality segmentation in brain MRI.
- [Taskonomy](https://github.com/StanfordVL/taskonomy) (`taskonomy`). A set of networks trained for different visual tasks, like Keypoint-Detection, Depth-Estimation, Reshading, etc. The initial idea for these networks was to find relationships between different visual tasks.
- [Slowfast](https://github.com/facebookresearch/pytorchvideo) (`pyvideo`). 
These models are state-of-the-art video classification models trained on the Kinetics 400 dataset, acessible through the torch.hub API.
- [CLIP](https://github.com/openai/CLIP) (`clip`). 
CLIP (Contrastive Language-Image Pre-Training) is a vision+language multimodal neural network trained on a variety of (image, text) pairs.
- [CorNet](https://github.com/dicarlolab/CORnet) (`cornet`). 
A set of neural networks whose structure is supposed to resemble the one of the ventral visual pathway and therefore implements more recurrent connections that are commonplace in the VVS.
- [Detectron2](https://github.com/facebookresearch/Detectron) (`detectron2`). 
Facebook AI Research's software system that implements state-of-the-art object detection algorithms, including Mask R-CNN. It covers models trained for object classification and detection such as instance, panoptic and keypoint detection.
- [VISSL](https://github.com/facebookresearch/vissl) (`vissl`). 
VISSL provides reference implementation of a large number of self-supervision approaches.

You can print the available models from every netset using the function `print_all_models()`. 

In [ ]:
from net2brain.feature_extraction import print_all_models

print_all_models()

You can also inspect the models available from a particular _netset_ using the function `print_netset_models()`:

In [ ]:
from net2brain.feature_extraction import print_netset_models

print_netset_models('pyvideo')

Or you can find a model by its name using the function `find_model_like()`:

In [ ]:
from net2brain.feature_extraction import find_model_like

find_model_like('resnet50')

## Using `FeatureExtractor` with a pretrained DNN

To extract the activations of a pretrained model from one of the netsets, you will first need to initialize the `FeatureExtractor` class, and provide the name of the model as well as the name of the _netset_. You can also determine which device to use to compute the extraction, in case you want to run it on GPUs.

In [ ]:
from net2brain.feature_extraction import FeatureExtractor

fx = FeatureExtractor(model='ResNet50', netset='standard', device='cpu')

__Net2Brain__ chooses by default from which layers of the model to extract the features from. You can inspect which layers are selected by default by calling the `layers_to_extract` attribute:

In [ ]:
fx.layers_to_extract

However, you can also select which layers to extract. If you would only want the activations from specific layer, for example layer 4, you can define this is the `FeatureExtractor` arguments:

In [ ]:
fx = FeatureExtractor(
    model='ResNet50', netset='standard', 
    layers_to_extract=['layer4'], 
    device='cpu'
  )

If you are not sure about the names of the layers that you could extract from a given model beyond the __Net2Brain__ default ones, you can always use the `get_all_layers()` method to get a print out of the possibilities:

In [ ]:
fx.get_all_layers()

To initialize the extraction, you have to call the method `extract()`. Using this method, you can specify how you want the activations to be stored using the `save_format` argument. Options are (1)`pt` or (2)`npz`, in which cases the activations are stored separately for each image in a tensor or array format (respectively), or (3)`dataset`, in which case the activations are stored in the format of the `Dataset` class of the [RSA toolbox](https://rsatoolbox.readthedocs.io/en/stable/).

You can also specify in which folder to store the activations using the `save_path` argument. By default the activations will be stored in a folder named `features` at the root of the project.

In [ ]:
images_path = 'Net2Brain/input_data/stimuli_data/78images'

fx = FeatureExtractor(model='ResNet50', netset='standard', device='cpu')
fts_datasets = fx.extract(
    dataset_path=images_path, save_format='dataset', save_path='features'
  )

If `dataset` is provided as the output format, the function returns a dictionary of a Dataset class with an entry for each of the layers specified:

In [ ]:
fts_datasets.keys()

Each entry is a `dataset` class:

In [ ]:
fts_datasets['layer1']

 These dataset classes are stored separetely in the output folder. You can load these classes back into your code using the `load_dataset` function from the rsatoolbox dataset like so: 

In [ ]:
from pathlib import Path
from rsatoolbox.data.dataset import load_dataset

filename = Path('features/ResNet50_layer1.hdf5')
layer1_dataset = load_dataset(filename, file_type='hdf5')
layer1_dataset

## Using `FeatureExtractor` with your own DNN

You can also use __Net2Brain__ with your own model. For this, you will need to provide to the `FeatureExtractor` the model, the transforms of the images, and the layers to extract.

Let us see an example using a custom Alex Net:

In [ ]:
from torchvision import models
from torchvision import transforms as T

# Define model and transforms
model = models.alexnet(pretrained=True)
#model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
transforms = T.Compose([
    T.Resize((224, 224)),  # transform images if needed
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

# Define extractor
layers = ['features.0', 'features.1']
fx = FeatureExtractor(
    model=model, transforms=transforms, layers_to_extract=layers, device='cpu'
)

# Run extractor
feats = fx.extract(
    dataset_path=images_path, save_format='dataset', save_path='features2'
  )

## Coming Soon!

- [ ] Create researcher-friendly taxonomy of DNNs
- [ ] Optional pretrained models argument
- [ ] Optimization of extraction speed
- [ ] Costumization of output states
- [ ] API documentation

### Your ideas?
- [ ] ?